# Direct Problem

the direct problem for the inc operator is the following:
Given a function $\gamma \in H(cc)$ find the function $\text{inc}\gamma \in H_\mathbb{S}(d)$

The problem is divided into 2 steps
1. find a function $\xi = \text{inc}\gamma \in H(cc)$ 
2. find pass to the space $\hat{\xi} \in H_\mathbb{S}(d)$

This is due to the fact that the elasticity complex is :
\begin{align}
H(cc) \to ^{inc}  H_\mathbb{S}(d)
\end{align}

The 2 problems can be formulated into the variational form:

1. find $\xi \in H(cc)$ function that minimizes the problem $$\text{min}_\xi \| \xi - \text{inc}\gamma \|^2 $$
2. find  $\hat{\xi} \in  H_\mathbb{S}(d)$ such that minimizes the problem $$\text{min}_{\nabla^S \hat{\xi} = 0 } \| \hat{\xi} - \xi \|^ 2$$

The above problems in variational form are:

1.  find $\xi \in H(cc)$ such that
$$ \langle \xi , \eta\rangle = \langle \text{inc}\gamma, \eta\rangle \quad \forall \eta \in H(cc)$$
2. find $(\hat{\xi} , p) \in H_\mathbb{S}(d)\times H(cc)$ such that
$$
\begin{cases}
\langle \hat{\xi}, \eta\rangle  &+& \langle \nabla^S \hat{\eta} , p \rangle &=&  \langle \xi, \eta\rangle \quad \forall \eta \in H_\mathbb{S}(d)\\
\langle \nabla^S \hat{\xi} , q \rangle  & & &=& 0 \forall q \in H(cc) \\
\end{cases}
$$




In [8]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

In [9]:
n = specialcf.normal(3) # normal vector to a surface of a tetrahedron

def norm(u, Mesh):
    # norm of a scalar, vector or matrix function
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh)) 

def Curl(u):
    # curl of a vector or matrix
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )

def Inc(u):
    # inc of a matrix
    return Curl((Curl(u)).trans)

P_n = OuterProduct(n,n)  # projection along normal direction
Q_n = Id(3) - OuterProduct(n,n) # projection along tangential direction
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) ) # corss product perator with the normal vector

def C(V): 
    # corss product perator with the generic vector
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


In [15]:


def distributive_inc(mesh , order=2,gamma = None, BND_inc_gamma = None,inc_gamma= None,draw = True, bonus_intorder = 15):
    """
    g is the grid function of the gamma living in HCurlCurl space
    mesh is the mesh 
    order is the order of the finite element space

    """

    fesHCurlCurl = HCurlCurl(mesh, order=order, dirichlet= ".*")

    ##########################################################
    #           Coefficient and grid functions               #
    ##########################################################

    # grid functions 
    gf_gamma = GridFunction(fesHCurlCurl) 
    gf_gamma.Set ( gamma, bonus_intorder=bonus_intorder, dual=True)

    gf_inc_g = GridFunction(fesHCurlCurl)
    gf_inc_g.Set(BND_inc_gamma, BND,  bonus_intorder=bonus_intorder, dual=True)

    ##########################################################
    #              Linear and Bilinear forms                 #
    ##########################################################

    u,v = fesHCurlCurl.TnT()

    #some geometrical objects we need
    n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
    t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
    t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
    e = specialcf.tangential(3,True)
    n1 = Cross( t1, e)
    n2 = Cross( t2, e) 

    # Mass matrix
    a = BilinearForm(fesHCurlCurl, symmetric=True)
    a += InnerProduct(u,v)*dx 

    # linear form induced by the metric gfG
    f = LinearForm(fesHCurlCurl)

    # thetrahedron inc part
    f += InnerProduct(gf_gamma.Operator("inc"),v)*dx        
    # faces part:
    f += ( InnerProduct(Q_n*n_cross_v, curl(gf_gamma).trans) + Cross(gf_gamma*n,n)*(curl(v)*n) )*dx(element_boundary=True)
    # Edges components: t'*v*C_n*n
    f += (gf_gamma[n1,e]*v[e,t1] - gf_gamma[n2,e]*v[e,t2])*dx(element_vb=BBND)
        
    ##########################################################
    #           Assemble, compute and print!                 #
    ##########################################################
    pre = Preconditioner(a, "local")
    a.Assemble()
    f.Assemble()

    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gf_inc_g.vec
    inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
    gf_inc_g.vec.data += inverse * r
    #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r

    if draw:
        print("error of my inc")
        Error = norm(BND_inc_gamma-gf_inc_g,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(order)+"-->"+str(Error) )
        Draw (gf_inc_g-BND_inc_gamma,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gf_inc_g,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (BND_inc_gamma, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
    return gf_inc_g

# pass from the HCurlCurl space to the Hsymdiv space, 


In [18]:
maxh = 0.15 
order = 3
mesh = Mesh(unit_cube.GenerateMesh(maxh=maxh))
draw = False      

# our gamma
cf_peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))
cf_gamma = CF ( (cf_peak, 0, 0 , 0,cf_peak,0,0,0,cf_peak), dims=(3,3))

# true inc(gamma)
cf_inc_gamma = CF( Inc(cf_gamma), dims=(3,3) )
with TaskManager():
    gf_inc_g = distributive_inc(mesh , order=order ,gamma = cf_gamma, BND_inc_gamma = cf_inc_gamma,inc_gamma= cf_inc_gamma,draw = draw, bonus_intorder = 15)


Draw (gf_inc_g,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
Draw (cf_inc_gamma,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)


CG NOT converged in 1000 iterations to residual 0.00010632259542583531


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

BaseWebGuiScene